<a href="https://colab.research.google.com/github/danjonesss/ResearchProject/blob/main/MNIST_Classifier_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install codecarbon

In [2]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,f1_score
from codecarbon import EmissionsTracker
import time
from random import randint
from sklearn import svm

In [ ]:
ET = EmissionsTracker()
svmET = EmissionsTracker()
num_classes = 10
input_shape = (28,28,1)

(x_train,y_train),(x_test,y_test)=keras.datasets.mnist.load_data()

x_train = x_train.astype("float32")/255
x_test = x_test.astype("float32")/255

x_train=np.expand_dims(x_train,-1)
x_test=np.expand_dims(x_test,-1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0],"train samples")
print(x_test.shape[0],"test samples")


y_train = keras.utils.to_categorical(y_train,num_classes)
y_test = keras.utils.to_categorical(y_test,num_classes)

ET.start()
StartTime = time.time()
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()
batch_size = 64
epochs = 24

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

history=model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1,verbose=0)
emissions: float = ET.stop()
EndTime = time.time() - StartTime

In [ ]:
score = model.evaluate(x_test, y_test, verbose=1)
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred,axis=1)
Y_test = np.argmax(y_test,axis=1)
cm = confusion_matrix(Y_test,y_pred)
print(cm)
accuracy = accuracy_score(Y_test,y_pred)
precision = precision_score(Y_test,y_pred,average="micro")
recall = recall_score(Y_test,y_pred,average="micro")
f1 = f1_score(Y_test,y_pred,average="micro")
print("")
print("Accuracy: ",accuracy)
print("Precision: ",precision)
print("Recall: ",recall)
print("F1 Score: ",f1)

In [ ]:
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)

y_train_svm = np.argmax(y_train, axis=1)
y_test_svm = np.argmax(y_test, axis=1)

svmET.start()
svm_StartTime = time.time()
svm_model = svm.SVC(kernel='linear')
svm_model.fit(x_train_flat, y_train_svm)
svm_EndTime = time.time() - svm_StartTime
svm_emissions: float = svmET.stop()

y_pred_svm = svm_model.predict(x_test_flat)
svm_cm = confusion_matrix(y_test_svm, y_pred_svm)

svm_accuracy = accuracy_score(y_test_svm, y_pred_svm)
svm_precision = precision_score(y_test_svm, y_pred_svm, average="micro")
svm_recall = recall_score(y_test_svm, y_pred_svm, average="micro")
svm_f1 = f1_score(y_test_svm, y_pred_svm, average="micro")

print(svm_cm)
print("\nSVM Accuracy: ", svm_accuracy)
print("SVM Precision: ", svm_precision)
print("SVM Recall: ", svm_recall)
print("SVM F1 Score: ", svm_f1)

In [ ]:
print("Test loss:", score[0])
print("Test accuracy:", score[1])
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')
plt.show()
print("")
print(f'NN Emissions: {emissions} kg')
print(f'SVM Emissions: {svm_emissions} kg')
print("")
print("NN Time taken: ", EndTime, "seconds")
print("SVM Time taken: ", svm_EndTime, "seconds")
print("")
print("NN Accuracy : ", accuracy*100, "%")
print("SVM Accuracy : ", svm_accuracy*100, "%")
print("")

In [ ]:
fig, ax1 = plt.subplots(1,1)
num = randint(0,9999)
ax1.imshow(x_test[num], cmap='gray', interpolation = 'none')
ax1.set_title('Prediction: {}, True: {}'.format(y_pred[num], Y_test[num]))
ax1.set_xticks([])
ax1.set_yticks([])

In [ ]:
misclassified_indices = np.where(Y_test != y_pred)[0]

fig, ax2 = plt.subplots(1,1)
num1 = np.random.choice(misclassified_indices)
ax2.imshow(x_test[num1], cmap='gray', interpolation = 'none')
ax2.set_title('Prediction: {}, True: {}'.format(y_pred[num1], Y_test[num1]))
ax2.set_xticks([])
ax2.set_yticks([])
